In [ ]:
import json
import matplotlib.pyplot as plt
import glob
import statistics

import pandas as pd
import seaborn as sns

from os.path import join
from os import walk, path

from DLBio.kwargs_translator import get_kwargs

In [ ]:
SRC_DIR = '../experiments/eval_custom_models'

In [ ]:
def load_json(file_path):
    with open(file_path, 'r') as file:
        out = json.load(file)
    return out

In [ ]:
folders = glob.glob(join(SRC_DIR, 'layer*'))

# get all logs and opts and save the relevant features in a DataFrame
data = []
for i,folder in enumerate(folders):
    log = load_json(join(folder, 'log.json'))
    opt = load_json(join(folder, 'opt.json'))
    
    if not bool(log):
        print(f'log_{i} is empty')
        continue
    
    # Params: 
    acc = log['val_acc'][-1]
    er = log['val_er'][-1]
    
    d = {}
    
    model_kw = get_kwargs(opt['model_kw'])
    d['num_layer'] =  int(model_kw.get('num_layer', [0])[0])
    d['init_dim'] = int(model_kw.get('init_dim', [0])[0])
    
    d['seed'] = opt['seed']    
    d['acc'] = acc    
    d['er'] = er
    
   
    
    data.append(d)
    
    

In [ ]:
df = pd.DataFrame(data)
key = ('acc', 'mean')

agg = {
        'seed' : 'min',
        'acc' : ['mean', 'min'],
        'init_dim' : 'first',
        'er' : 'mean'
      }
df.groupby(['num_layer', 'init_dim']).agg(agg).sort_values(by=[key], ascending=False)



In [ ]:
fig, axes = plt.subplots(2,2, figsize=(20, 20))
sns.boxplot(ax=axes[0, 0], x="num_layer", y="acc",data=df)
sns.stripplot(ax=axes[0, 1],x="num_layer", y="acc", data=df, hue="init_dim")

sns.boxplot(ax=axes[1, 0], x="num_layer", y="er",data=df)
sns.stripplot(ax=axes[1, 1], x="num_layer", y="er", data=df, hue="init_dim")